# **Creating an OpenMDAO Component Wrapper for a VSP Model**

The OpenVSP installation here, specific to Colab, is adapted from the Ubuntu instructions provided on the [OpenVSP site](http://openvsp.org/wiki/doku.php?id=ubuntu_instructions).

First, install required packages:

In [ ]:
!apt-get update && \
 apt-get install git cmake libxml2-dev libfltk1.3-dev g++ libcpptest-dev \
  libjpeg-dev libglm-dev libeigen3-dev libcminpack-dev libglew-dev swig \
  doxygen graphviz python3-venv

Next, set up the directory structure and clone the OpenVSP source:

In [ ]:
!mkdir -p repo build buildlibs && \
 git clone --depth=1 https://github.com/OpenVSP/OpenVSP.git repo

Prepare the build files for the libraries and build them:

In [ ]:
!cd buildlibs && \
 cmake \
  -DVSP_USE_SYSTEM_LIBXML2=true \
  -DVSP_USE_SYSTEM_FLTK=true \
  -DVSP_USE_SYSTEM_GLM=true \
  -DVSP_USE_SYSTEM_GLEW=true \
  -DVSP_USE_SYSTEM_CMINPACK=true \
  -DVSP_USE_SYSTEM_LIBIGES=false \
  -DVSP_USE_SYSTEM_EIGEN=false \
  -DVSP_USE_SYSTEM_CODEELI=false \
  -DVSP_USE_SYSTEM_CPPTEST=false \
  ../repo/Libraries -DCMAKE_BUILD_TYPE=Release && \
 make -j 8

Build OpenVSP, which may take 10-15 minutes on Colab. Generate a zip folder that we can use to install the Python packages from:

In [ ]:
!cd build && \
 cmake ../repo/src \
  -DVSP_LIBRARY_PATH=/content/buildlibs \
  -DCMAKE_BUILD_TYPE=Release && \
 make -j 8 && \
 make package

Build and install the OpenVSP Python packages. The current version of OpenVSP may have to be updated:

In [ ]:
import os
os.environ["VSP_VER"] = "3.21.2"

!cd /content/build/_CPack_Packages/Linux/ZIP/OpenVSP-${VSP_VER}-Linux/python && \
  pip install -r requirements.txt && \
  cd .. && \
  cp vspaero vspscript vspslicer /usr/local/bin

Run a simple test to be sure the packages are installed correctly:

In [ ]:
import openvsp

# **Incorporating OpenVSP into OpenMDAO**
## Install OpenMDAO


In [ ]:
pip install openmdao jdc

# Sample Code
This is an OpenMDAO component wrapper for a VSP model of a modified CRM (eCRM-001) that is used to demonstrate the computation and use of stability derivatives in a design problem. This code is based on a sample provided during OpenMDAO RevHack 2020.

First, obtain the CRM:

In [ ]:
!wget -c https://raw.githubusercontent.com/OpenMDAO/RevHack2020/master/problems/oas_stability_derivs/eCRM-001.1_wing_tail.vsp3

Import required modules:

In [ ]:
import pickle
import itertools

import numpy as np

import openmdao.api as om

import openvsp as vsp
import degen_geom

import jdc # To permit class definitions to be spread across multiple cells

Unable to import mpi4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.
Unable to import petsc4py. Parallel processing unavailable.


Create a new subclass based on OpenMDAO [ExplicitComponent](https://openmdao.org/twodocs/versions/latest/features/core_features/defining_components/explicitcomp.html). Set some [options](https://openmdao.org/twodocs/versions/latest/features/core_features/defining_components/options.html) with the names of the geometries in the VSP project that we want to work with. The reduced option is helpful for testing purposes.

In [ ]:
class VSPeCRM(om.ExplicitComponent):

    def initialize(self):
        self.options.declare('horiz_tail_name', default='Tail',
                             desc="Name of the horizontal tail in the vsp model.")
        self.options.declare('vert_tail_name', default='VerticalTail',
                             desc="Name of the vertical tail in the vsp model.")
        self.options.declare('wing_name', default='Wing',
                             desc="Name of the wing in the vsp model.")
        self.options.declare('reduced', default=False,
                             desc="When True, output reduced meshes instead of full-size ones. "
                             "Running with a smaller mesh is of value when debugging.")

Load the OpenVSP project from a VSP3 file and find the IDs of the relevant geometries.

Set up inputs with initial values, and outputs with units and 3-dimensional shapes. The finite difference approximation method for partial derivatives is used.

In [ ]:
    %%add_to VSPeCRM
    def setup(self):
        options = self.options
        horiz_tail_name = options['horiz_tail_name']
        vert_tail_name = options['vert_tail_name']
        wing_name = options['wing_name']
        reduced = options['reduced']

        # Read the geometry.
        vsp_file = 'eCRM-001.1_wing_tail.vsp3'
        vsp.ReadVSPFile(vsp_file)

        self.wing_id = vsp.FindGeomsWithName(wing_name)[0]
        self.horiz_tail_id = vsp.FindGeomsWithName(horiz_tail_name)[0]
        self.vert_tail_id = vsp.FindGeomsWithName(vert_tail_name)[0]

        self.add_input('wing_cord', val=59.05128,)
        self.add_input('vert_tail_area', val=2295.)
        self.add_input('horiz_tail_area', val=6336.)

        # Shapes are pre-determined.
        if reduced:
            self.add_output('wing_mesh', shape=(6, 9, 3), units='inch')
            self.add_output('vert_tail_mesh', shape=(5, 5, 3), units='inch')
            self.add_output('horiz_tail_mesh', shape=(5, 5, 3), units='inch')
        else:
            # Note: at present, OAS can't handle this size.
            self.add_output('wing_mesh', shape=(23, 33, 3), units='inch')
            self.add_output('vert_tail_mesh', shape=(33, 9, 3), units='inch')
            self.add_output('horiz_tail_mesh', shape=(33, 9, 3), units='inch')

        self.declare_partials(of='*', wrt='*', method='fd')

Compute the OpenVSP meshes. Using the previously located geometries, set the values of the VSP parameters to the initial input values from `setup()`. Then, reset the OpenVSP model to a blank slate with `Update()`.

Compute the degenerate geometry representation for the OpenVSP components, and obtain the measurements.

The purpose of this RevHack exercise was to use [OpenAeroStruct](https://github.com/mdolab/OpenAeroStruct), so update the meshes to work with it.

In [ ]:
    %%add_to VSPeCRM
    def compute(self, inputs, outputs, discrete_inputs=None, discrete_outputs=None):
        # set values
        vsp.SetParmVal(self.vert_tail_id, "TotalArea", "WingGeom", inputs['vert_tail_area'][0])
        vsp.SetParmVal(self.horiz_tail_id, "TotalArea", "WingGeom", inputs['horiz_tail_area'][0])
        vsp.SetParmVal(self.wing_id, "TotalChord", "WingGeom", inputs['wing_cord'][0])

        vsp.Update()
        vsp.Update()  # just in case..

        # run degen geom to get measurements
        dg:degen_geom.DegenGeomMgr = vsp.run_degen_geom(set_index=vsp.SET_ALL)
        obj_dict = {p.name:p for p in dg.get_all_objs()}

        # pull measurements out of degen_geom api
        degen_obj: degen_geom.DegenGeom = obj_dict[self.options['wing_name']]
        wing_cloud = self.vsp_to_point_cloud(degen_obj)

        degen_obj: degen_geom.DegenGeom = obj_dict[self.options['horiz_tail_name']]
        horiz_cloud = self.vsp_to_point_cloud(degen_obj)

        degen_obj: degen_geom.DegenGeom = obj_dict[self.options['vert_tail_name']]
        vert_cloud = self.vsp_to_point_cloud(degen_obj)

        # VSP outputs wing outer mold lines at points along the span.
        # Reshape to (chord, span, dimension)
        wing_cloud = wing_cloud.reshape((45, 33, 3), order='F')
        horiz_cloud = horiz_cloud.reshape((33, 9, 3), order='F')
        vert_cloud = vert_cloud.reshape((33, 9, 3), order='F')

        # Meshes have upper and lower surfaces, so we average the z (or y for vertical).
        wing_pts = wing_cloud[:23, :, :]
        wing_pts[1:-1, :, 2] = 0.5 * (wing_cloud[-2:-23:-1, :, 2] + wing_pts[1:-1, :, 2])
        horiz_tail_pts = horiz_cloud[:17, :, :]
        horiz_tail_pts[1:-1, :, 2] = 0.5 * (horiz_cloud[-2:-17:-1, :, 2] + horiz_tail_pts[1:-1, :, 2])
        vert_tail_pts = vert_cloud[:17, :, :]
        vert_tail_pts[1:-1, :, 1] = 0.5 * (vert_cloud[-2:-17:-1, :, 1] + vert_tail_pts[1:-1, :, 1])

        # Reduce the mesh size for testing. (See John Jasa's recommendations in the docs.)
        if self.options['reduced']:
            wing_pts = wing_pts[:, ::4, :]
            wing_pts = wing_pts[[0, 4, 8, 12, 16, 22], ...]
            horiz_tail_pts = horiz_tail_pts[::4, ::2, :]
            vert_tail_pts = vert_tail_pts[::4, ::2, :]

        # Flip around so that FEM normals yield positive areas.
        wing_pts = wing_pts[::-1, ::-1, :]
        horiz_tail_pts = horiz_tail_pts[::-1, ::-1, :]
        vert_tail_pts = vert_tail_pts[::-1, ::-1, :]

        # outputs go here
        outputs['wing_mesh'] = wing_pts
        outputs['vert_tail_mesh'] = horiz_tail_pts
        outputs['horiz_tail_mesh'] = vert_tail_pts

Convert an OpenVSP degenerate geometry to a [NumPy N-dimensional array](https://numpy.org/doc/stable/reference/generated/numpy.ndarray.html) of points using [itertools](https://docs.python.org/3/library/itertools.html).

In [ ]:
    %%add_to VSPeCRM
    def vsp_to_point_cloud(self, degen_obj: degen_geom.DegenGeom)->np.ndarray:
        npts = degen_obj.surf.num_pnts
        n_xsecs = degen_obj.surf.num_secs

        points = np.empty((npts * n_xsecs, 3))
        points[:, 0] = list(itertools.chain.from_iterable(degen_obj.surf.x))
        points[:, 1] = list(itertools.chain.from_iterable(degen_obj.surf.y))
        points[:, 2] = list(itertools.chain.from_iterable(degen_obj.surf.z))

        return points

Set up and run the model.


In [ ]:
if __name__ == "__main__":

    vsp_comp = VSPeCRM(horiz_tail_name="Tail",
                       vert_tail_name="VerticalTail",
                       wing_name="Wing",
                       reduced=True)

    p = om.Problem()

    model = p.model

    p.model.add_subsystem("vsp_comp", vsp_comp)

    p.setup()

    p.run_model()

Serialize and save the data for later use in OpenAeroStruct.

In [ ]:
    data = {}
    for item in ['wing_mesh', 'vert_tail_mesh', 'horiz_tail_mesh']:
        data[item] = p.get_val(f"vsp_comp.{item}", units='m')

    # Save the meshes in a pickle. These will become the undeformed baseline meshes in
    # OpenAeroStruct.
    with open('baseline_meshes_reduced.pkl', 'wb') as f:
        pickle.dump(data, f)

    print('done')

done
